# Chapter 03. 미분과 최적화 (Optimization)
AI가 스스로 데이터를 학습하고 오차를 줄여나가는 수치 최적화의 원리를 이해하고, 실무에서 사용하는 고등 옵티마이저의 메커니즘을 분석합니다.

### 1. 연쇄 법칙(Chain Rule)과 자동 미분 (Step 1)
- **목표**: 합성함수의 미분 원리를 이해하고 역전파(Backpropagation)의 기초 확립.
- **Deep Dive**: 계산 그래프(Computational Graph)를 통한 오차의 역방향 전파 수식 증명.
- **Engineering**: `Autograd` 엔진의 기본 설계 원칙 및 메모리 효율적인 미분값 관리 기법.

### 2. 경사하강법(Gradient Descent) 직접 구현 (Step 2)
- **목표**: 손실 함수(Loss Function)의 최솟값을 찾아가는 파라미터 업데이트 루프 설계.
- **Deep Dive**: MSE(Mean Squared Error)의 편미분 유도 및 학습률(Learning Rate)에 따른 수렴 속도 분석.
- **Engineering**: NumPy 벡터화 연산을 활용하여 수천 개의 파라미터를 한 번에 업데이트하는 성능 최적화.

### 3. 고등 옵티마이저: Adam & RMSProp (Step 3)
- **목표**: 데이터의 특성에 따라 학습률을 스스로 조절하는 Adaptive Optimizer 이해.
- **Deep Dive**: 지수 이동 평균(EMA)을 이용한 모멘텀(Momentum)과 학습률 감쇠 수식 분석.
- **Engineering**: 학습 초기 발산을 막기 위한 하이퍼파라미터 튜닝 및 옵티마이저 선택 가이드.

# Chapter 03-1. 연쇄 법칙(Chain Rule)과 자동 미분

신경망은 수많은 층(Layer)이 겹쳐진 거대한 합성함수입니다. 각 층의 가중치가 최종 오차(Loss)에 미치는 영향을 계산하기 위해 **연쇄 법칙**을 사용하여 미분값을 뒤에서부터 앞으로 전달합니다.

> **Key Point**: $\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$ 원리를 이용하면 아무리 복잡한 신경망도 부분적인 미분의 곱으로 표현 가능합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. [데이터 준비] 모델이 학습할 가상의 정답 데이터를 만듭니다.
# np.random.seed(42): 무작위 결과를 고정하여 실행할 때마다 같은 데이터가 나오게 합니다.
np.random.seed(42)

# X: 공부 시간 (0~2시간 사이의 랜덤값 100개), y: 실제 성적 (y = 3X + 4 + 노이즈)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# 2. [파라미터 초기화] 우리가 찾아야 할 정답(w=3, b=4)을 처음엔 모르는 상태로 설정합니다.
# w: 가중치(Slope), b: 편향(Intercept). 처음엔 무작위 점(random)에서 시작합니다.
w = np.random.randn(1, 1)
b = np.zeros((1, 1))

# 3. [하이퍼파라미터 설정] 학습의 속도와 횟수를 결정합니다.
learning_rate = 0.1  # 한 번의 학습에서 얼마나 큰 보폭으로 이동할지 결정하는 수치
epochs = 500         # 전체 데이터를 몇 번 반복해서 학습할지 결정

# 4. [학습 루프] 반복문을 돌며 점진적으로 오차를 줄여나갑니다.
for epoch in range(epochs):
    
    # --- Step A. 예측 (Forward Pass) ---
    # 현재의 w와 b를 사용하여 예측값(y_pred)을 계산합니다. (일차방정식: y = wX + b)
    y_pred = X.dot(w) + b
    
    # --- Step B. 오차 계산 (Loss Calculation) ---
    # 실제값(y)과 예측값(y_pred)의 차이를 제곱하여 평균을 낸 MSE(Mean Squared Error)를 구합니다.
    # 이 Loss 값이 0에 가까워지는 것이 학습의 최종 목표입니다.
    loss = np.mean((y_pred - y)**2)
    
    # --- Step C. 미분 (Backward Pass - Gradient Calculation) ---
    # 수식: Loss를 w와 b로 각각 편미분하여 오차가 커지는 '방향'을 찾습니다.
    # dw: 가중치 w를 얼마나 수정해야 하는지에 대한 변화량
    # db: 편함 b를 얼마나 수정해야 하는지에 대한 변화량
    # (2/100)은 미분 공식에서 내려오는 지수와 데이터 개수(N=100)를 나눈 값입니다.
    dw = (2/100) * X.T.dot(y_pred - y)  # X.T는 행렬 곱을 위해 X를 전치(Transpose)한 것 dot은 행렬곱셈 메서드
    db = (2/100) * np.sum(y_pred - y)    # 편향은 단순히 오차들의 합에 비례하여 조정됨
    
    # --- Step D. 최적화 (Parameter Update) ---
    # 찾은 방향(dw, db)의 반대 방향으로 learning_rate만큼 이동하여 w와 b를 갱신합니다.
    # 마이너스(-)를 쓰는 이유는 경사(Gradient)가 낮은 곳으로 내려가기 위함입니다.
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    # 100번 학습할 때마다 현재 상태를 출력하여 잘 되고 있는지 확인합니다.
    if epoch % 100 == 0:
        print(f"Epoch {epoch:3d}: Loss = {loss:.4f}, w = {w[0][0]:.2f}, b = {b[0][0]:.2f}")

# 5. [결과 확인] 학습이 끝난 후 최종적으로 찾은 수식을 출력합니다.
print("-" * 50)
print(f"최종 결과 - 예측된 수식: y = {w[0][0]:.2f}X + {b[0][0]:.2f}")
print(f"실제 정답 - 목표 수식: y = 3.00X + 4.00")

"""
[ 🛠️ 엔지니어링 분석 리포트 ]

1. 시간 복잡도: O(Epochs * N)
   - 데이터 개수(N)만큼 행렬 연산을 수행하고 이를 Epoch 횟수만큼 반복하므로 선형적으로 증가합니다.

2. 실행 환경 및 성능 최적화:
   - 본 코드는 NumPy의 벡터화 연산(dot product)을 사용하여 Python의 일반 for 루프보다 수천 배 빠릅니다.
   - 데이터가 천만 건 이상으로 늘어날 경우, 전체 데이터를 한 번에 계산하는 'Batch' 방식 대신 'Mini-batch' 방식으로의 전환이 필요합니다.

3. 원인 분석 (Engineering Insight):
   - 만약 Loss가 줄어들지 않고 발산한다면, learning_rate가 너무 커서 최적의 지점을 지나쳐버리는 현상입니다.
"""

Epoch   0: Loss = 49.8165, w = 1.49, b = 1.36
Epoch 100: Loss = 0.8069, w = 2.80, b = 4.18
Epoch 200: Loss = 0.8066, w = 2.77, b = 4.21
Epoch 300: Loss = 0.8066, w = 2.77, b = 4.22
Epoch 400: Loss = 0.8066, w = 2.77, b = 4.22
--------------------------------------------------
최종 결과 - 예측된 수식: y = 2.77X + 4.22
실제 정답 - 목표 수식: y = 3.00X + 4.00


"\n[ 🛠️ 엔지니어링 분석 리포트 ]\n\n1. 시간 복잡도: O(Epochs * N)\n   - 데이터 개수(N)만큼 행렬 연산을 수행하고 이를 Epoch 횟수만큼 반복하므로 선형적으로 증가합니다.\n\n2. 실행 환경 및 성능 최적화:\n   - 본 코드는 NumPy의 벡터화 연산(dot product)을 사용하여 Python의 일반 for 루프보다 수천 배 빠릅니다.\n   - 데이터가 천만 건 이상으로 늘어날 경우, 전체 데이터를 한 번에 계산하는 'Batch' 방식 대신 'Mini-batch' 방식으로의 전환이 필요합니다.\n\n3. 원인 분석 (Engineering Insight):\n   - 만약 Loss가 줄어들지 않고 발산한다면, learning_rate가 너무 커서 최적의 지점을 지나쳐버리는 현상입니다.\n"

### 💡 디버깅 가이드: 학습률(Learning Rate) 설정 에러

> **[원인 분석]**
> 1. **Learning Rate가 너무 클 때**: 최솟값을 지나쳐 발산(Exploding)하며 Loss가 `inf` 또는 `NaN`이 됨.
> 2. **Learning Rate가 너무 작을 때**: 최솟값에 도달하기 전에 학습이 끝나거나, Local Minimum에 갇혀 성능이 정체됨.

> **[재발 방지 팁]**
> 1. 학습 초기에는 `0.1`, `0.01`, `0.001` 등 로그 스케일로 시도하며 적정 범위를 찾으세요.
> 2. `np.isnan(loss)` 조건을 추가하여 발산 시 학습을 조기 종료(Early Stopping)하는 로직을 검토하세요.

# Chapter 03-3. 고등 옵티마이저: Adam & RMSProp

단순 경사하강법(SGD)의 한계를 극복하기 위해 현대 딥러닝에서는 **Adam**을 가장 표준적으로 사용합니다.

1. **Momentum (관성)**: 가던 방향으로 계속 가려는 성질을 이용해 Local Minimum을 탈출합니다.
2. **RMSProp (적응적 학습률)**: 변화가 적은 파라미터는 크게, 변화가 큰 파라미터는 작게 학습률을 조절하여 안정적으로 수렴시킵니다.
3. **Adam**: 위 두 가지 장점을 결합한 알고리즘으로, 대부분의 딥러닝 문제에서 기본 옵션으로 선택됩니다.

In [3]:
import numpy as np

# [데이터 준비] 이전 실습과 동일한 선형 데이터
np.random.seed(42)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# [파라미터 초기화]
w = np.random.randn(1, 1)
b = np.zeros((1, 1))

# [Adam 하이퍼파라미터 설정]
learning_rate = 0.01
beta1 = 0.9   # 모멘텀(관성) 계수
beta2 = 0.999 # 적응적 학습률(지수 이동 평균) 계수
epsilon = 1e-8 # 0으로 나누는 것을 방지하기 위한 아주 작은 값

# 관성과 속도를 저장할 변수 초기화 (모양은 파라미터와 동일하게)
m_w, v_w = 0, 0
m_b, v_b = 0, 0

# [학습 루프]
for epoch in range(1, 301):
    # 1. Forward Pass (예측)
    y_p = X.dot(w) + b
    
    # 2. Gradient 계산 (이전 단계에서 배운 X.T.dot 활용)
    dw = (2/100) * X.T.dot(y_p - y)
    db = (2/100) * np.sum(y_p - y)
    
    # 3. Adam 업데이트 로직 (핵심 기능)
    # Step 1: 모멘텀 업데이트 (가던 방향 기억)
    m_w = beta1 * m_w + (1 - beta1) * dw
    m_b = beta1 * m_b + (1 - beta1) * db
    
    # Step 2: 개별 학습률 조절 (기울기 크기 기억)
    v_w = beta2 * v_w + (1 - beta2) * (dw**2)
    v_b = beta2 * v_b + (1 - beta2) * (db**2)
    
    # Step 3: 편향 보정 (초기 학습 안정화)
    m_w_hat = m_w / (1 - beta1**epoch)
    m_b_hat = m_b / (1 - beta1**epoch)
    v_w_hat = v_w / (1 - beta2**epoch)
    v_b_hat = v_b / (1 - beta2**epoch)
    
    # Step 4: 파라미터 최종 업데이트
    # 분모의 sqrt(v_hat)이 학습률을 자동으로 조절해줌
    w = w - learning_rate * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
    b = b - learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)

    if epoch % 100 == 0:
        loss = np.mean((X.dot(w) + b - y)**2)
        print(f"Epoch {epoch}: Loss = {loss:.4f}")

print(f"\nAdam 최종 결과 - w: {w[0][0]:.2f}, b: {b[0][0]:.2f}")

"""
[ 🛠️ 상세 주석 및 기능 설명 ]
- m_w (Momentum): 기울기의 방향성을 유지하여 진동을 줄임.
- v_w (Velocity): 기울기의 제곱값을 누적하여 파라미터별로 학습 보폭을 조절.
- epsilon: 분모가 0이 되어 에러가 발생하는 것을 방지하는 안전장치.

- 시간 복잡도: O(Epochs * N) (추가 연산량이 적으면서 수렴 속도는 훨씬 빠름)
"""

Epoch 100: Loss = 26.6722
Epoch 200: Loss = 13.0609
Epoch 300: Loss = 5.9769

Adam 최종 결과 - w: 2.34, b: 2.36


'\n[ 🛠️ 상세 주석 및 기능 설명 ]\n- m_w (Momentum): 기울기의 방향성을 유지하여 진동을 줄임.\n- v_w (Velocity): 기울기의 제곱값을 누적하여 파라미터별로 학습 보폭을 조절.\n- epsilon: 분모가 0이 되어 에러가 발생하는 것을 방지하는 안전장치.\n\n- 시간 복잡도: O(Epochs * N) (추가 연산량이 적으면서 수렴 속도는 훨씬 빠름)\n'

### 💡 디버깅 가이드: 옵티마이저 선택 장애

> **[원인 분석]**
> 초보 엔지니어는 어떤 옵티마이저를 쓸지 고민하다 시간을 낭비하곤 합니다. 하지만 데이터의 특성(희소성, 노이즈 등)에 따라 최적의 알고리즘은 다를 수 있습니다.

> **[재발 방지 팁]**
> 1. **고민될 땐 Adam**: 대부분의 경우 Adam이 가장 빠르고 안정적입니다.
> 2. **정교한 튜닝은 SGD + Momentum**: 최종 모델의 정확도를 극대화할 때는 Adam보다 단순한 방식이 더 세밀하게 조정될 때가 있습니다.
> 3. **학습 곡선 모니터링**: Loss가 춤을 추듯 요동친다면 `learning_rate`를 낮추거나 `beta1` 값을 조정해 보세요.